#### Erstellung des Datensatzes

Der Datensatz wird durch Aufnahmen der eigenen Hand erstellt. Hierbei wurden insgesamt 200 Bilder mithilfe einer Webcam aufgenommen, die später auch für das Spiel getestet werden. Die Daten werden außerdem durch Augmentation-Techniken erweitert. Dabei wird jedes Bild um zusätzliche 250 Bilder erweitert, wobei folgende Augmentation-Methoden angewandt werden:

* Drehung
* Anpassung von Helligkeit und Kontrast
* Skalierung und Verschiebung
* Hinzufügen von Rauschen
* Horizontale Spiegelung

Schließlich werden all diese Bilder in einem Ordner zusammengeführt.

In [1]:
import cv2
import os
import random
import numpy as np
import shutil

In [2]:
# Erhalte letzte Bildnummer im Verzeichnis
def get_last_image_index(save_directory):
    image_files = [f for f in os.listdir(save_directory) if os.path.isfile(os.path.join(save_directory, f))]
    if not image_files:
        return 0

    indexes = [int(f.split('.')[0]) for f in image_files]
    return max(indexes)

# Überprüfe, ob das Bild schwarz ist
def is_black_image(frame, threshold=30):
    # Berechne die durchschnittliche Helligkeit des Bildes
    avg_brightness = int(frame.mean())

    # Wenn die durchschnittliche Helligkeit unter dem Schwellenwert liegt, ist das Bild größtenteils schwarz
    return avg_brightness < threshold

# Bilder speichern
def capture_images(camera_index, num_images, save_directory):
    # Öffne die hintere Kamera
    cap = cv2.VideoCapture(camera_index)

    if not cap.isOpened():
        print("Kamera konnte nicht geöffnet werden.")
        return

    # Erstelle das Verzeichnis, falls es nicht existiert
    os.makedirs(save_directory, exist_ok=True)

    # Wir lesen das erste Bild ohne es zu speichern
    _, _ = cap.read()

    last_index = get_last_image_index(save_directory)
    start_index = last_index + 1

    for i in range(start_index - 1, start_index + num_images - 1):
        # Bilder aufnehmen
        ret, frame = cap.read()
        if not ret:
            print("Fehler beim Erfassen des Bildes.")
            break

        # Bild anzeigen
        cv2.imshow(f"Bild {i}", frame)

        # Warte 1 Sekunde zwischen den Bildern
        cv2.waitKey(1000)

        # Überprüfe, ob das Bild schwarz ist
        if not is_black_image(frame):
            # Bild speichern
            file_name = os.path.join(save_directory, f"{i}.jpg")
            cv2.imwrite(file_name, frame)

    # Kamera freigeben und alle Fenster schließen
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# Initialisierung der Variablen für die Datensammlung
camera_index = 0  # 0 für die Vorderkamera, 1 für die Rückkamera (falls vorhanden)
num_images = 101  # Anzahl der Bilder, die aufgenommen werden sollen (ignoriert erstes Bild)
save_directory = "./dataset/img"

In [20]:
# Speicherung der offenen Hand in verschiedenen Positionen
capture_images(camera_index, num_images, save_directory)

In [ ]:
# Speicherung der geschlossenen Hand in verschiedenen Positionen
capture_images(camera_index, num_images, save_directory)

#### Augmentation

In [9]:
def flip_images(input_folder, output_folder, num_augmentations_per_image=1, num_random_images=50):
    # Erstelle den Ausgabeordner, falls er nicht existiert
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Liste aller Dateien im Eingabeordner
    image_files = os.listdir(input_folder)

    # Zufällige Bilder auswählen
    random_images = random.sample(image_files, min(num_random_images, len(image_files)))

    for image_file in random_images:
        input_path = os.path.join(input_folder, image_file)
        output_prefix, ext = os.path.splitext(image_file)

        # Lade das Bild
        image = cv2.imread(input_path)

        # Horizontales Spiegeln (Horizontal Flip)
        for i in range(num_augmentations_per_image):
            augmented_image = cv2.flip(image, 1)  # 1 für horizontales Spiegeln
            output_file = f"{output_prefix}_aug_{i+1}{ext}"
            output_path = os.path.join(output_folder, output_file)
            cv2.imwrite(output_path, augmented_image)


In [ ]:
input_folder = "dataset/img"
output_folder = "dataset/aug"
num_augmentations_per_image = 1
num_random_images = 50

flip_images(input_folder, output_folder, num_augmentations_per_image, num_random_images)

In [19]:
def augment_rotation(image, angle_range=(-15, 15)):
    angle = random.uniform(angle_range[0], angle_range[1])
    rows, cols = image.shape[:2]
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    return cv2.warpAffine(image, M, (cols, rows))

def augment_brightness_contrast(image, brightness_range=(-30, 30), contrast_range=(0.5, 1.5)):
    brightness = random.uniform(brightness_range[0], brightness_range[1])
    contrast = random.uniform(contrast_range[0], contrast_range[1])
    return np.clip(image * contrast + brightness, 0, 255).astype(int)

def augment_scaling_translation(image, scale_range=(0.8, 1.2), translation_range=(-20, 20)):
    rows, cols = image.shape[:2]
    scale_factor = random.uniform(scale_range[0], scale_range[1])
    scaled_image = cv2.resize(image, (int(cols * scale_factor), int(rows * scale_factor)))
    
    translation_x = random.randint(translation_range[0], translation_range[1])
    translation_y = random.randint(translation_range[0], translation_range[1])
    
    M = np.float32([[1, 0, translation_x], [0, 1, translation_y]])
    return cv2.warpAffine(scaled_image, M, (cols, rows))

def augment_noise(image, mean=0, stddev=10):
    noise = np.random.normal(mean, stddev, image.shape).astype(int)
    return np.clip(image + noise, 0, 255).astype(int)


def augment_images(input_folder, output_folder, num_augmentations_per_image=1, num_random_images=50):
    # Erstelle den Ausgabeordner, falls er nicht existiert
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Liste aller Dateien im Eingabeordner
    image_files = os.listdir(input_folder)

    # Zufällige Bilder auswählen
    random_images = random.sample(image_files, min(num_random_images, len(image_files)))

    # Index für die Benennung der Bilder
    index = 1

    for image_file in random_images:
        input_path = os.path.join(input_folder, image_file)
        output_prefix, ext = os.path.splitext(image_file)

        # Lade das Bild
        image = cv2.imread(input_path)

        # Generiere 50 Bilder für jede Augmentation-Funktion
        for i in range(num_augmentations_per_image):
            augmented_image = cv2.flip(image, 1)  # 1 für horizontales Spiegeln
            output_file = f"{output_folder}/{index}.jpg"
            cv2.imwrite(output_file, augmented_image)
            index += 1

            augmented_image = augment_rotation(image)
            output_file = f"{output_folder}/{index}.jpg"
            cv2.imwrite(output_file, augmented_image)
            index += 1

            augmented_image = augment_brightness_contrast(image)
            output_file = f"{output_folder}/{index}.jpg"
            cv2.imwrite(output_file, augmented_image)
            index += 1

            augmented_image = augment_scaling_translation(image)
            output_file = f"{output_folder}/{index}.jpg"
            cv2.imwrite(output_file, augmented_image)
            index += 1

            augmented_image = augment_noise(image)
            output_file = f"{output_folder}/{index}.jpg"
            cv2.imwrite(output_file, augmented_image)
            index += 1

In [ ]:
input_folder = "dataset/img"
output_folder = "dataset/aug"
num_augmentations_per_image = 1
num_random_images = 50

augment_images(input_folder, output_folder, num_augmentations_per_image, num_random_images)

Zusammenführung der beiden Ordner

In [ ]:
def merge_and_rename_folders(folder1_path, folder2_path, destination_folder_path):
    # Ensure the destination folder exists
    os.makedirs(destination_folder_path, exist_ok=True)

    # Counter for the new file names
    counter = 1

    # Iterate through the first folder and copy the images
    for file in os.listdir(folder1_path):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            source = os.path.join(folder1_path, file)
            target = os.path.join(destination_folder_path, f'{counter}.jpg')
            shutil.copy(source, target)
            counter += 1

    # Iterate through the second folder and copy the images
    for file in os.listdir(folder2_path):
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            source = os.path.join(folder2_path, file)
            target = os.path.join(destination_folder_path, f'{counter}.jpg')
            shutil.copy(source, target)
            counter += 1

    print("Merging and renaming completed.")


In [ ]:
# Paths to the two folders with images
folder1_path = 'dataset/aug'
folder2_path = 'dataset/img'
destination_folder_path = 'dataset/img_merged'

# Call the function with the provided folder paths
merge_and_rename_folders(folder1_path, folder2_path, destination_folder_path)